In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from collections import Counter

In [2]:
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
url2 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
df_usconf = pd.read_csv(url,error_bad_lines = False)
df_usdead = pd.read_csv(url2,error_bad_lines = False)
pd.options.display.max_columns = None

In [4]:
df_usconf.loc[df_usconf['Province_State'] == 'California'].head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20
191,84006001,US,USA,840,6001.0,Alameda,California,US,37.646294,-121.892927,"Alameda, California, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,2,2,2,3,5,5,7,11,11,18,27,30,38,48,68,71,118,135,135,178,220,220,270,283,313,313,359,443,443,566,588,634,674,674,766,806,845,888,888,964,1007,1063,1114,1164,1193,1241,1242,1352,1403,1439,1470,1500,1533,1568,1603,1636,1706,1749,1776,1809,1863,1917,1961,2023,2064,2101,2133,2178,2234,2367,2351,2392,2457,2522,2560,2609,2708,2760,2847,2874,2986,3049,3097,3195,3289,3390,3470,3515,3548,3641,3725,3805,3805,3945,3985,4033,4232,4216,4217,4320,4373,4481,4533,4638,4702,4924,4924,5007,5267,5275,5382,5493,5493,5670
192,84006003,US,USA,840,6003.0,Alpine,California,US,38.596786,-119.822359,"Alpine, California, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
193,84006005,US,USA,840,6005.0,Amador,California,US,38.445831,-120.656960,"Amador, California, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,2,2,2,2,2,2,2,2,2,3,3,3,0,3,3,3,3,7,7,7,7,7,7,7,7,7,7,7,7,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,9,9,9,9,9,10,10,10,10,10,10,10,10,10,10,10,10,10,19,10,10,10,10,10,10,10,10,10,10,10,10,10,10,11,12,12,12,12,13,13,22,16,19,20,20,20,21
194,84006007,US,USA,840,6007.0,Butte,California,US,39.667278,-121.600525,"Butte, California, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,4,5,5,5,5,5,8,8,9,10,11,11,11,12,13,13,13,14,14,14,15,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,17,17,18,18,18,18,18,20,20,20,22,22,22,22,24,26,30,31,34,34,34,34,40,40,40,44,44,44,51,51,55,60,65,65,65,68,71,73,73,74,74,74,82,88,90,90,94,94,94,111,119,127,133,143,143,143
195,84006009,US,USA,840,6009.0,Calaveras,California,US,38.205371,-120.552913,"Calaveras, California, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,3,3,3,3,3,3,3,3,4,4,5,5,5,5,8,8,8,8,9,9,11,11,12,12,12,12,12,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,14,15,15,15,15,15,15,15,15,16,19,19,19,19,19,21,25,25,26,26,26,26,26,26,28,28,28,28,29,29,29,32,32,32


In [ ]:
columns_to_remove = ['UID', 'code3', 'FIPS']
US_confirmed_daily_increase = df_usconf.copy()[Counter(df_usconf.columns.tolist()) - Counter(columns_to_remove)]
US_dead_daily_increase = df_usdead.copy()[Counter(df_usdead.columns.tolist()) - Counter(columns_to_remove)]

US_confirmed_daily_increase.iloc[:,8:] = US_confirmed_daily_increase.iloc[:,8:].diff(axis=1).fillna(0).astype('int')
US_dead_daily_increase.iloc[:,9:] = US_dead_daily_increase.iloc[:,9:].diff(axis=1).fillna(0).astype('int')

In [ ]:
BA_counties = ['Alameda','Contra Costa','Marin','Napa','San Francisco','San Mateo','Santa Clara','Solano','Sonoma']
LA_counties =['Ventura','San Bernadio', 'Riverside', 'Los Angeles', 'Orange']

CA_daily_increase = US_confirmed_daily_increase[(US_confirmed_daily_increase['Province_State']=='California')]
BA_daily_increase = US_confirmed_daily_increase[(US_confirmed_daily_increase['Province_State']=='California') & (US_confirmed_daily_increase['Admin2'].isin(BA_counties))]
LA_daily_increase = US_confirmed_daily_increase[(US_confirmed_daily_increase['Province_State']=='California') & (US_confirmed_daily_increase['Admin2'].isin(LA_counties))]

CA_transpose = CA_daily_increase.copy().transpose().rename(columns = pd.Series(CA_daily_increase['Admin2']))[8:]
BA_transpose = BA_daily_increase.copy().transpose().rename(columns = pd.Series(BA_daily_increase['Admin2']))[8:]
LA_transpose = LA_daily_increase.copy().transpose().rename(columns = pd.Series(LA_daily_increase['Admin2']))[8:]

US_transpose
US_transpose_dates = US_confirmed_daily_increase.copy().transpose().rename(columns = pd.Series(US_confirmed_daily_increase['Combined_Key']))[8:]
US_header = US_confirmed_daily_increase.copy().iloc[:,:8]

In [ ]:
CA_transpose.to_csv('X:\AC\Documents\Datasets\CA_daily_increments.csv')
df_usconf.to_csv(r'X:\AC\Documents\Datasets\US_confimed_raw.csv')
US_transpose_dates.to_csv(r'X:\AC\Documents\Datasets\US_confirmed_dates.csv')
US_header.to_csv(r'X:\AC\Documents\Datasets\US_confirmed_headers.csv')